In [ ]:
# default_exp machine_learning

# Machine Learning

> Apply simple machine learning algorithms to a measures table

In [ ]:
#hide
from nbdev.showdoc import *

## Overview

The gamba library contains several wrapper methods for commonly applied machine learning methods...

## Methods

### Clustering

Clustering techniques have been popular in early literature, but continue to be applied across a number of problems. The gamba library contains wrapper methods around the scikit-learn library, which has implementations of a number of clustering algorithms.

### Neural Networks

Neural networks are a powerful way to learn high dimensional patterns in transaction data sets. The gamba library contains limited neural network capabilities, but provide fully annotated methods so you can copy them into your own workflows and extend them as needed.

## Plotting

Some machine learning outputs favour plotting over raw tabular presentation. An example of this is a hierarchical clustering dendrogram, which is much more intuitive than a cluster membership report.